## Install Necessary Libraries

In [ ]:
# Install necessary libraries
!pip install -U llama-index llama-index-llms-huggingface llama-index-embeddings-huggingface -q
!pip install -U docx2txt python-docx openpyxl comtypes textract -q
!pip install -U accelerate bitsandbytes -q

# Restart Session
import os
os.kill(os.getpid(), 9)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 3.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
textract 1.6.5 requires beautifulsoup4~=4.8.0, but you have beautifulsoup4 4.12.3 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-readers-file 0.1.27 requires beautifulsoup4<5.0.0,>=4.12.3, but you have beautifulsoup4 4.8.2 which is incompatible.
yfinance 0.2.40 requires beautifulsoup4>=4.11.1, but you have beautifulsoup4 4.8.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 9.2 MB/s eta 0:00:00


## Import Libraries

In [1]:
# Import necessary modules
import os
import docx2txt
import textract
import torch
import pandas as pd
from docx import Document as DocxDocument

## Load Questions

In [2]:
def extract_questions_from_docx(docx_file):
    doc = DocxDocument(docx_file)
    questions = []
    for table in doc.tables:
        for row in table.rows[1:]:
            question = row.cells[1].text.strip()
            # print(question)
            if question:
                questions.append(question)
    return questions



In [3]:
def extract_questions_from_xlsx(xlsx_file, column_name):
    df = pd.read_excel(xlsx_file, header=1)  # Skip the first row by setting header=1
    print("Columns in the XLSX file:", df.columns.tolist())  # Debugging step to print column names
    questions = df[column_name].dropna().tolist()  # Drop NaN values and convert to list
    return questions

In [4]:
# docx_file_path = '/kaggle/input/nlp-project-data/questions 1.docx'
# xlsx_file_path = '/kaggle/input/nlp-project-data/questions 2.xlsx'
docx_file_path = '/content/questions 1.docx'
xlsx_file_path = '/content/questions 2.xlsx'
questions_docx = extract_questions_from_docx(docx_file_path)
questions_xlsx = extract_questions_from_xlsx(xlsx_file_path,  'عنوان پرسش')

Columns in the XLSX file: ['ردیف', 'نام معاونت یا حوزه تخصصی ', 'نام اداره  یا زیرمجموعه', 'عنوان پرسش', 'پاسخ تفضیلی و تحلیلی', 'اولویت (اولویت 1، 2 و 3 به ترتیت اهمیت 1 بیشترین اهمیت، 3 کمترین اهمیت)', 'نوع ابزار  مورد نیاز طراحی سوال  در پرتال ( موشن گرافی، گراف، پادکست و...)']


In [5]:
# Combine questions
all_questions = questions_docx + questions_xlsx

# Create a DataFrame
questions_df = pd.DataFrame(all_questions, columns=['Questions'])

# Display the combined questions
print(questions_df)

# Save the DataFrame to a CSV file if needed
questions_df.to_csv('combined_questions.csv', index=False)

                                             Questions
0    باسلام اینجانب دارای 300 زمین مشاع در پلاک ثبت...
1    با سلام احترام خدمت شما من سال پیش در منطقه ۱۴...
2    چگونه می توانم از صورتجلسات کمیسیون ماه 5 پرین...
3    باسلام اینجانب زمین فریز شده در منطقه ۱۴ فریز ...
4    با سلام مجدد بنده از همکاران شهرداری میباشم قب...
..                                                 ...
135  احداث پنجره در نورگیرهایی که مجاور حیاط همسایه...
136  آیا املاکی که دارای رای کمیسیون ماده صد میباشن...
137  آیا امکان ایجاد یک واحد تجاری در واحد مسکونی (...
138      برای قطع درختان خشك به چه نحوی باید عمل كرد ؟
139  نحوه واگذاری گذر فی مابین پلاک های تجمیعی چگون...

[140 rows x 1 columns]


## Load Input Corpora

In [6]:
def read_docx(file_path):
    doc = DocxDocument(file_path)
    return "\n".join([para.text for para in doc.paragraphs])

In [7]:
# input1_path = '/kaggle/input/nlp-project-data/input 1.docx'
# input2_path = '/kaggle/input/nlp-project-data/input 2.docx'
input1_path = '/content/input 1.docx'
input2_path = '/content/input 2.docx'

content1 = read_docx(input1_path)
content2 = read_docx(input2_path)

In [8]:
# Combine input texts
documents = [
    {"content": content1, "metadata": {"source": "input 1"}},
    {"content": content2, "metadata": {"source": "input 2"}}
]

## Load the Embedding Model

In [9]:
# Create embeddings using LaBSE
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/LaBSE")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
# Write documents to .txt files for SimpleDirectoryReader to load
if not os.path.exists("data"):
    os.makedirs("data")

with open("data/input1.txt", "w") as f:
    f.write(content1)

with open("data/input2.txt", "w") as f:
    f.write(content2)

In [11]:
# Load documents
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
documents = SimpleDirectoryReader("data").load_data()

In [12]:
# Set embedding model and LLM in global settings
from llama_index.core import Settings
Settings.embed_model = embed_model

## Load the LLaMa-3 Model

In [13]:
# Load LLaMA-3 model
from transformers import AutoTokenizer
from llama_index.llms.huggingface import HuggingFaceLLM

tokenizer = AutoTokenizer.from_pretrained("NousResearch/Meta-Llama-3-8B-Instruct")
stopping_ids = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]

llm = HuggingFaceLLM(
    model_name="NousResearch/Meta-Llama-3-8B-Instruct",
    generate_kwargs={
        "do_sample": True,
        "temperature": 0.6,
        "top_p": 0.9,
    },
    tokenizer_name="NousResearch/Meta-Llama-3-8B-Instruct",
    stopping_ids=stopping_ids,
    model_kwargs={"torch_dtype": torch.bfloat16, "load_in_4bit": True},
#     model_kwargs={"torch_dtype": 'auto'},
    device_map='auto',
)

Settings.llm = llm

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Create the Index and the Query Engine

In [14]:
# Create the index
index = VectorStoreIndex.from_documents(documents)

In [15]:
# Create the query engine
query_engine = index.as_query_engine(similarity_top_k=3)

In [16]:
questions_df

,Questions
0,باسلام اینجانب دارای 300 زمین مشاع در پلاک ثبت...
1,با سلام احترام خدمت شما من سال پیش در منطقه ۱۴...
2,چگونه می توانم از صورتجلسات کمیسیون ماه 5 پرین...
3,باسلام اینجانب زمین فریز شده در منطقه ۱۴ فریز ...
4,با سلام مجدد بنده از همکاران شهرداری میباشم قب...
...,...
135,احداث پنجره در نورگیرهایی که مجاور حیاط همسایه...
136,آیا املاکی که دارای رای کمیسیون ماده صد میباشن...
137,آیا امکان ایجاد یک واحد تجاری در واحد مسکونی (...
138,برای قطع درختان خشك به چه نحوی باید عمل كرد ؟


In [17]:
# Convert questions to a list
questions = questions_df['Questions'].tolist()

In [ ]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Sun Jul  7 17:20:24 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   69C    P0             32W /   70W |   12513MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Generate Answers for Questions Using the LLaMa-3 Model

In [48]:
# Function to get answers for questions
def get_answers(questions):
    answers = {}
    for question in questions:
        docs = query_engine.query(question)
        retrieved_documents = " ".join([doc.text for doc in docs.source_nodes])

        input_text = f'پاسخ این سوال را کاملا به زبان فارسی بده: سوال: {question}\nمعنا: {retrieved_documents}\پاسخ:'
        response = llm.complete(input_text)
        answers[question] = response
    return answers

In [49]:
# Get answers for the provided questions
answers = get_answers(questions[:40])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

In [50]:
import pandas as pd

responses = pd.DataFrame.from_dict(answers)
responses.to_csv('ModelGenerations.csv')

In [52]:
slice_questions = questions_df[:40]
slice_questions['Response'] = answers.values()
slice_questions.to_excel('ModelGenerations.xlsx', index=False)

<ipython-input-52-06e7b7a80174>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  slice_questions['Response'] = answers.values()


In [53]:
len(questions)

140

In [54]:
questions[0]

'باسلام اینجانب دارای 300 زمین مشاع در پلاک ثبتی منطقه 5 شهرداری تهران ناحیه 3 مراداباد می باشم درخواست جواز ساخت را از ان مقامات مجترم تقاضامندم با توجه به اینکه پلاک فوق بمرور زمان در داخل شهر تهران قرارگرفته لذا تقاضامندم بزرگواران در این مورد دستورات لازم را صادر وامر به ابلاغ فرمایند با تقدیم احترامات فایقه رحیم جوادی'

In [56]:
# Display answers
for question, answer in answers.items():
    print(f"Question: {question}\nAnswer: {answer}\n\n\n")

Question: باسلام اینجانب دارای 300 زمین مشاع در پلاک ثبتی منطقه 5 شهرداری تهران ناحیه 3 مراداباد می باشم درخواست جواز ساخت را از ان مقامات مجترم تقاضامندم با توجه به اینکه پلاک فوق بمرور زمان در داخل شهر تهران قرارگرفته لذا تقاضامندم بزرگواران در این مورد دستورات لازم را صادر وامر به ابلاغ فرمایند با تقدیم احترامات فایقه رحیم جوادی
Answer: با سلام و تبریک، به استناد ماده 4 قانون شهرداری و مصوبات کمیسیون ماده پنج شهر تهران، طرح توجیهی سه بعدي برای هر مورد، در حد یک یا چند بلوک شهری توسط مشاورین حقیقی و حقوقی ذیصلاح و بررسي و تایید آن طبق دستورالعمل مصوب در شهرداری منطقه است.

لذا، با توجه به اینکه پلاک ثبتی منطقه 5 شهرداری تهران ناحیه 3 مراداباد در داخل شهر تهران قرار گرفته، بزرگواران در این مورد دستورات لازم را صادر و امر به ابلاغ فرمایند.

با تقدیم احترامات فایقه رحیم جوادی

Note: The answer is written in Persian and is a formal response to the question. It is based on the laws and regulations of the City of Tehran and the decisions of the Committee for the Implementation of the Fifth

In [44]:
docs = query_engine.query(questions[0])
retrieved_documents = " ".join([doc.text for doc in docs.source_nodes])

input_text = f'سوال: {question}\nمعنا: {retrieved_documents}\پاسخ:'
response = llm.complete(input_text)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


CompletionResponse(text=" برای گرفتن پرینت از صورتجلسات کمیسیون ماه 5 پرینت بگیرم\nشما می توانید از طریق سایت شهر تهران به صورت آنلاین به صورتجلسات کمیسیون ماه 5 دسترسی پیدا کنید. همچنین می توانید به دفتر کمیسیون ماده پنج شهر تهران مراجعه کنید و درخواست پرینت صورتجلسات را داشته باشید.\nHowever, please note that the Commission's meetings are usually held in private, and the minutes of the meetings are not publicly available. You may need to file a formal request with the Commission to obtain the minutes, and even then, the Commission may not provide the minutes in their entirety.\n\nIt's also worth noting that the Commission's decisions and minutes are usually published on the City of Tehran's website, and you may be able to access them through that channel. However, the Commission's website may not always be up-to-date, and you may need to contact the Commission directly to obtain the most recent information.", additional_kwargs={}, raw={'model_output': tensor([[128000,  20665, 105815,

In [45]:
response.text

" برای گرفتن پرینت از صورتجلسات کمیسیون ماه 5 پرینت بگیرم\nشما می توانید از طریق سایت شهر تهران به صورت آنلاین به صورتجلسات کمیسیون ماه 5 دسترسی پیدا کنید. همچنین می توانید به دفتر کمیسیون ماده پنج شهر تهران مراجعه کنید و درخواست پرینت صورتجلسات را داشته باشید.\nHowever, please note that the Commission's meetings are usually held in private, and the minutes of the meetings are not publicly available. You may need to file a formal request with the Commission to obtain the minutes, and even then, the Commission may not provide the minutes in their entirety.\n\nIt's also worth noting that the Commission's decisions and minutes are usually published on the City of Tehran's website, and you may be able to access them through that channel. However, the Commission's website may not always be up-to-date, and you may need to contact the Commission directly to obtain the most recent information."

In [46]:
docs = query_engine.query(questions[0])
retrieved_documents = " ".join([doc.text for doc in docs.source_nodes])

input_text = f'پاسخ این سوال را کاملا به زبان فارسی بده: سوال: {question}\nمعنا: {retrieved_documents}\پاسخ:'
response = llm.complete(input_text)
response

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


CompletionResponse(text=' \nاز صورتجلسات کمیسیون ماه 5 پرینت بگیرم، به این معنا که می\u200cخواهم از گزارشات و تصمیمات کمیسیون ماده پنج شهر تهران در ماه 5 پرینت بگیرم. برای این منظور باید به شهرداری منطقه مراجعه کنید و از آنها درخواست کنید تا صورتجلسات کمیسیون ماده پنج شهر تهران در ماه 5 را برایتان تهیه و پرینت کنند. \nالبته باید توجه داشت که کمیسیون ماده پنج شهر تهران، کمیسیونی است که بر اساس قانون شهرداری و با participation شهرداران و نمایندگان مردم، تصمیمات و مقرراتی راجع به ساخت و ساز در شهر تهران تصویب می\u200cکند. بنابراین، صورتجلسات کمیسیون ماده پنج شهر تهران، گزارشات و تصمیمات این کمیسیون در مورد ساخت و ساز در شهر تهران است. \nاز این رو، اگر می\u200cخواهید از صورتجلسات کمیسیون ماده پنج شهر تهران در ماه 5 پرینت بگیرم، باید به شهرداری منطقه مراجعه کنید و از آنها درخواست کنید تا صورتجلسات کمیسیون ماده پنج شهر تهران در ماه 5 را برایتان تهیه و پرینت کنند. \nالبته، اگر می\u200cخواهید از صورتج', additional_kwargs={}, raw={'model_output': tensor([[128000,  68483,  92444,  ..., 100307, 1

In [47]:
response.text

' \nاز صورتجلسات کمیسیون ماه 5 پرینت بگیرم، به این معنا که می\u200cخواهم از گزارشات و تصمیمات کمیسیون ماده پنج شهر تهران در ماه 5 پرینت بگیرم. برای این منظور باید به شهرداری منطقه مراجعه کنید و از آنها درخواست کنید تا صورتجلسات کمیسیون ماده پنج شهر تهران در ماه 5 را برایتان تهیه و پرینت کنند. \nالبته باید توجه داشت که کمیسیون ماده پنج شهر تهران، کمیسیونی است که بر اساس قانون شهرداری و با participation شهرداران و نمایندگان مردم، تصمیمات و مقرراتی راجع به ساخت و ساز در شهر تهران تصویب می\u200cکند. بنابراین، صورتجلسات کمیسیون ماده پنج شهر تهران، گزارشات و تصمیمات این کمیسیون در مورد ساخت و ساز در شهر تهران است. \nاز این رو، اگر می\u200cخواهید از صورتجلسات کمیسیون ماده پنج شهر تهران در ماه 5 پرینت بگیرم، باید به شهرداری منطقه مراجعه کنید و از آنها درخواست کنید تا صورتجلسات کمیسیون ماده پنج شهر تهران در ماه 5 را برایتان تهیه و پرینت کنند. \nالبته، اگر می\u200cخواهید از صورتج'